In [60]:
from helper import *
pd.set_option('display.max_rows', 100)

In [93]:
# load preds
preds_2025 = pd.read_csv('./data/clean/predictions_2025.csv')[['player', 'pos', 'ppg_pred', 'ppg_rank_2025', 'ppg_rank_2024', 'pff_grade_2024']]

# load rankings
rankings_2025 = pd.read_csv('./data/rankings/adp_5-1.csv')

def clean_rankings(rankings):
    # combine firstName and lastName
    rankings['player'] = rankings['firstName'] + ' ' + rankings['lastName']

    # drop cols
    rankings = rankings.drop(columns=['id', 'firstName', 'lastName', 'teamName', 'slotName', 'lineupStatus'])

    # remove first 2 chars from positionRank col
    rankings['positionRank'] = rankings['positionRank'].str[2:]

    # rename positionRank to adp_rank_2025
    rankings = rankings.rename(columns={'positionRank': 'adp_rank_2025'})

    # drop players with null adp_rank_2025
    rankings = rankings[rankings['adp_rank_2025'].notnull()]

    # cast adp_rank_2025 to int
    rankings['adp_rank_2025'] = rankings['adp_rank_2025'].astype(int)

    return rankings

rankings_2025 = clean_rankings(rankings_2025)

In [94]:
# combine preds and rankings
adp = pd.merge(preds_2025, rankings_2025, on='player', how='left')

# calculate rank_diff
adp['rank_diff'] = adp['adp_rank_2025'] - adp['ppg_rank_2025']

adp.head(10)

,player,pos,ppg_pred,ppg_rank_2025,ppg_rank_2024,pff_grade_2024,adp,projectedPoints,adp_rank_2025,byeWeek,rank_diff
0,Lamar Jackson,QB,20.971642,1.0,1,94.0,36.1,355.8,2.0,NaN,1.0
1,Josh Allen,QB,18.648424,2.0,2,91.8,33.3,358.5,1.0,NaN,-1.0
2,Patrick Mahomes,QB,18.130830,3.0,13,85.8,78.8,320.3,6.0,NaN,3.0
3,Jared Goff,QB,16.489944,9.0,8,81.8,104.2,298.5,13.0,NaN,4.0
4,Bo Nix,QB,17.433376,6.0,9,76.4,95.3,312.5,10.0,NaN,4.0
5,Jayden Daniels,QB,17.145458,7.0,6,89.6,41.3,353.5,3.0,NaN,-4.0
6,Joe Burrow,QB,16.511486,8.0,3,93.9,49.4,329.5,5.0,NaN,-3.0
7,Ja'Marr Chase,WR,15.430802,1.0,1,85.8,1.1,260.2,1.0,NaN,0.0
8,Bijan Robinson,RB,15.391518,1.0,4,92.8,3.7,274.7,2.0,NaN,1.0
9,Jalen Hurts,QB,17.548178,5.0,5,76.3,48.6,345.1,4.0,NaN,-1.0


In [95]:
def show_top_players(pos, num_players, adp=adp):
    """
    Show the top players for a given position based on the predictions.
    """

    # filter the predictions for the specified position
    pos_preds = adp.query('pos == @pos').copy()[['player', 'adp_rank_2025', 'ppg_rank_2025', 'rank_diff', 'ppg_pred']]
    
    # sort by predicted points and select the top players
    top_players = pos_preds.sort_values(by='adp_rank_2025', ascending=True).reset_index(drop=True)

    return top_players.head(num_players).T

In [96]:
show_top_players('QB', 36)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
player,Josh Allen,Lamar Jackson,Jayden Daniels,Jalen Hurts,Joe Burrow,Patrick Mahomes,Baker Mayfield,Caleb Williams,Kyler Murray,Bo Nix,Justin Fields,Brock Purdy,Jared Goff,Justin Herbert,Drake Maye,Jordan Love,C.J. Stroud,Dak Prescott,Trevor Lawrence,Michael Penix,Tua Tagovailoa,Matthew Stafford,Bryce Young,Geno Smith,Anthony Richardson,Sam Darnold,Russell Wilson,Aaron Rodgers,Daniel Jones,Joe Flacco,Derek Carr,Kirk Cousins,Brandon Allen,Taylor Heinicke,Kenny Pickett,Will Levis
adp_rank_2025,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,21.0,22.0,24.0,25.0,26.0,27.0,28.0,30.0,32.0,35.0,36.0,37.0,39.0,40.0,40.0,42.0,42.0
ppg_rank_2025,2.0,1.0,7.0,5.0,8.0,3.0,11.0,19.0,4.0,6.0,27.0,17.0,9.0,10.0,21.0,26.0,22.0,20.0,25.0,62.0,12.0,14.0,31.0,16.0,35.0,18.0,15.0,29.0,28.0,34.0,23.0,13.0,61.0,54.0,59.0,44.0
rank_diff,-1.0,1.0,-4.0,-1.0,-3.0,3.0,-4.0,-11.0,5.0,4.0,-16.0,-5.0,4.0,4.0,-6.0,-10.0,-5.0,-2.0,-6.0,-41.0,10.0,10.0,-6.0,10.0,-8.0,10.0,15.0,3.0,7.0,2.0,14.0,26.0,-21.0,-14.0,-17.0,-2.0
ppg_pred,18.648424,20.971642,17.145458,17.548178,16.511486,18.13083,16.157238,14.12396,17.686892,17.433376,12.525399,14.31722,16.489944,16.309322,13.344249,12.562955,13.177473,14.078869,12.766508,4.191972,15.394548,14.682054,10.822092,14.329371,9.521683,14.191064,14.439679,11.369932,11.993955,9.701075,13.124017,14.991928,4.208695,5.154788,4.454111,6.866893


In [97]:
show_top_players('RB', 50)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
player,Saquon Barkley,Bijan Robinson,Jahmyr Gibbs,Christian McCaffrey,Derrick Henry,De'Von Achane,Jonathan Taylor,Bucky Irving,Josh Jacobs,Kyren Williams,Chase Brown,Breece Hall,James Cook,Joe Mixon,Chuba Hubbard,Kenneth Walker III,Alvin Kamara,James Conner,David Montgomery,Aaron Jones,Tony Pollard,Najee Harris,Jaylen Warren,D'Andre Swift,Isiah Pacheco,Brian Robinson Jr.,Jordan Mason,Zach Charbonnet,Rhamondre Stevenson,Javonte Williams,Tyjae Spears,Tank Bigsby,Trey Benson,Isaac Guerendo,Ray Davis,Rachaad White,Tyler Allgeier,Jerome Ford,Rico Dowdle,Braelon Allen,Austin Ekeler,J.K. Dobbins,Jaylen Wright,Blake Corum,Will Shipley,Justice Hill,Roschon Johnson,Nick Chubb,MarShawn Lloyd,Jaleel McLaughlin
adp_rank_2025,1.0,2.0,3.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,15.0,16.0,18.0,19.0,20.0,22.0,23.0,26.0,27.0,28.0,30.0,32.0,33.0,34.0,35.0,36.0,37.0,40.0,41.0,42.0,43.0,44.0,46.0,47.0,48.0,49.0,51.0,54.0,55.0,56.0,57.0,58.0,60.0,62.0,64.0,64.0,66.0,69.0
ppg_rank_2025,6.0,1.0,3.0,19.0,5.0,2.0,11.0,4.0,7.0,13.0,18.0,9.0,14.0,10.0,16.0,15.0,8.0,12.0,17.0,20.0,21.0,22.0,33.0,24.0,38.0,31.0,32.0,29.0,28.0,46.0,35.0,40.0,67.0,42.0,41.0,27.0,39.0,37.0,34.0,63.0,25.0,23.0,72.0,106.0,121.0,70.0,64.0,48.0,111.0,51.0
rank_diff,-5.0,1.0,0.0,-14.0,1.0,5.0,-3.0,5.0,3.0,-2.0,-6.0,4.0,1.0,6.0,2.0,4.0,12.0,10.0,6.0,6.0,6.0,6.0,-3.0,8.0,-5.0,3.0,3.0,7.0,9.0,-6.0,6.0,2.0,-24.0,2.0,5.0,20.0,9.0,12.0,17.0,-9.0,30.0,33.0,-15.0,-48.0,-61.0,-8.0,0.0,16.0,-45.0,18.0
ppg_pred,13.873262,15.391518,14.541001,10.72642,14.124535,14.609208,12.907993,14.438104,13.680642,12.803769,10.85105,13.009959,12.635178,12.929785,12.030789,12.486574,13.23697,12.804311,11.772822,10.319377,10.076751,9.690212,7.411744,9.376688,6.956335,8.063169,7.800839,8.608443,8.645573,6.394256,7.147609,6.82544,4.870065,6.595346,6.672292,9.037267,6.889465,7.04252,7.154991,5.215256,9.292506,9.58503,4.539722,2.919223,2.454502,4.625513,5.151001,6.298222,2.748875,6.094292


In [100]:
show_top_players('WR', 80)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79
player,Ja'Marr Chase,Justin Jefferson,CeeDee Lamb,Puka Nacua,Malik Nabers,Nico Collins,Amon-Ra St. Brown,Brian Thomas,Drake London,A.J. Brown,Ladd McConkey,Tee Higgins,Tyreek Hill,Jaxon Smith-Njigba,Garrett Wilson,Terry McLaurin,D.J. Moore,Davante Adams,Mike Evans,Rashee Rice,Xavier Worthy,DeVonta Smith,Jameson Williams,Zay Flowers,Courtland Sutton,Rome Odunze,Jordan Addison,Jaylen Waddle,Calvin Ridley,Chris Godwin,Chris Olave,Deebo Samuel,Jauan Jennings,Jerry Jeudy,Ricky Pearsall,Jakobi Meyers,Jayden Reed,George Pickens,Khalil Shakir,Stefon Diggs,Brandon Aiyuk,Cooper Kupp,Josh Downs,Darnell Mooney,Christian Kirk,Michael Pittman Jr.,Rashid Shaheed,Rashod Bateman,Keon Coleman,Jalen McMillan,Quentin Johnston,Xavier Legette,Romeo Doubs,Adam Thielen,Keenan Allen,Cedric Tillman,Jalen Coker,Alec Pierce,Wan'Dale Robinson,Jalen Tolbert,Dontayvion Wicks,DeAndre Hopkins,Michael Wilson,Adonai Mitchell,Darius Slayton,Andrei Iosivas,Amari Cooper,Dyami Brown,Tyler Lockett,Troy Franklin,Tutu Atwell,Mike Williams,KaVontae Turpin,Diontae Johnson,Jordan Whittington,Nick Westbrook-Ikhine,Tre Tucker,Devaughn Vele,Kayshon Boutte,Marquez Valdes-Scantling
adp_rank_2025,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,18.0,19.0,20.0,21.0,22.0,24.0,25.0,26.0,28.0,29.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,42.0,43.0,44.0,45.0,46.0,49.0,50.0,51.0,52.0,53.0,55.0,58.0,59.0,60.0,64.0,65.0,67.0,68.0,70.0,71.0,72.0,73.0,74.0,75.0,76.0,79.0,81.0,82.0,83.0,84.0,85.0,86.0,88.0,90.0,92.0,93.0,96.0,97.0,98.0,100.0,101.0,102.0,105.0,106.0
ppg_rank_2025,1.0,2.0,4.0,8.0,3.0,11.0,5.0,14.0,10.0,6.0,12.0,16.0,7.0,27.0,20.0,22.0,30.0,9.0,13.0,15.0,31.0,18.0,46.0,21.0,38.0,42.0,28.0,37.0,48.0,19.0,41.0,43.0,40.0,34.0,62.0,32.0,35.0,33.0,51.0,24.0,53.0,23.0,29.0,52.0,63.0,45.0,54.0,64.0,39.0,50.0,47.0,57.0,55.0,44.0,25.0,97.0,49.0,60.0,59.0,87.0,76.0,61.0,71.0,80.0,78.0,83.0,58.0,127.0,75.0,69.0,90.0,95.0,84.0,88.0,96.0,77.0,85.0,67.0,104.0,107.0
rank_diff,0.0,0.0,-1.0,-4.0,2.0,-5.0,2.0,-6.0,-1.0,4.0,-1.0,-4.0,6.0,-13.0,-5.0,-6.0,-12.0,10.0,7.0,6.0,-9.0,6.0,-21.0,5.0,-10.0,-13.0,3.0,-5.0,-15.0,15.0,-6.0,-7.0,-3.0,4.0,-23.0,8.0,7.0,10.0,-7.0,21.0,-7.0,26.0,21.0,-1.0,-11.0,8.0,1.0,-6.0,20.0,10.0,17.0,8.0,12.0,24.0,45.0,-26.0,23.0,13.0,15.0,-12.0,0.0,18.0,10.0,2.0,5.0,1.0,27.0,-41.0,13.0,21.0,2.0,-2.0,12.0,9.0,2.0,23.0,16.0,35.0,1.0,-1.0
ppg_pred,15.430802,14.144032,13.800525,12.532546,13.831751,12.240119,13.785197,11.761965,12.362638,12.749806,11.902663,11.210387,12.565837,10.045144,10.970756,10.575887,9.928565,12.420608,11.838449,11.666686,9.552837,11.133986,7.51526,10.656632,8.118458,7.823677,10.025332,8.231745,7.333409,11.011156,7.838877,7.606443,7.884178,9.265081,5.698667,9.442998,9.090008,9.31807,6.858729,10.296821,6.816072,10.48875,9.988104,6.853219,5.636343,7.521384,6.765176,5.384951,7.934302,6.907146,7.44135,5.974635,6.307727,7.550484,10.281774,3.493767,6.987817,5.869624,5.884559,3.830622,4.434016,5.797473,4.66311,4.246012,4.394306,4.020998,5.89011,2.519899,4.558681,4.759383,3.726696,3.520405,3.953953,3.824567,3.52035,4.43094,3.891242,5.201673,3.218084,3.12948


In [99]:
show_top_players('TE', 36)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
player,Brock Bowers,Trey McBride,George Kittle,Sam LaPorta,T.J. Hockenson,Jonnu Smith,Travis Kelce,Mark Andrews,Evan Engram,David Njoku,Tucker Kraft,Dalton Kincaid,Isaiah Likely,Jake Ferguson,Dallas Goedert,Kyle Pitts,Brenton Strange,Zach Ertz,Hunter Henry,Mike Gesicki,Pat Freiermuth,Cade Otton,Dalton Schultz,Cole Kmet,Ja'Tavion Sanders,Juwan Johnson,Theo Johnson,Tyler Conklin,Noah Gray,Tyler Higbee,Taysom Hill,Ben Sinnott,Will Dissly,Noah Fant,Luke Schoonmaker,Dawson Knox
adp_rank_2025,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,10.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,26.0,27.0,28.0,31.0,33.0,35.0,36.0,37.0,38.0,39.0,42.0,43.0,44.0,45.0
ppg_rank_2025,1.0,3.0,2.0,5.0,12.0,8.0,4.0,6.0,15.0,7.0,13.0,17.0,22.0,21.0,9.0,23.0,38.0,18.0,14.0,24.0,16.0,10.0,25.0,19.0,20.0,28.0,29.0,32.0,33.0,30.0,11.0,66.0,36.0,27.0,42.0,41.0
rank_diff,0.0,-1.0,1.0,-1.0,-7.0,-2.0,3.0,2.0,-5.0,5.0,0.0,-3.0,-7.0,-5.0,8.0,-5.0,-19.0,2.0,7.0,-2.0,7.0,14.0,1.0,8.0,8.0,3.0,4.0,3.0,3.0,7.0,27.0,-27.0,6.0,16.0,2.0,4.0
ppg_pred,12.4706,9.688758,10.625777,8.389908,6.389687,7.176982,9.469428,8.239586,5.819709,7.198138,6.334902,5.520925,4.895417,4.941911,6.861399,4.883747,3.197683,5.360552,5.839956,4.651441,5.536574,6.495208,4.592924,5.348976,5.088995,4.443986,4.428705,4.207585,4.188764,4.322243,6.451742,1.607394,3.381752,4.525823,2.750379,2.750592


In [13]:
teams_2024 = pd.read_csv('./data/pff/team/2024.csv')
teams_2024['blocking grade'] = (teams_2024['Pass Blocking Grade'] + teams_2024['Run Blocking Grade']).div(2)
teams_2024 = teams_2024[['Tm', 'Points For', 'Points Against', 'blocking grade', 'Pass Blocking Grade', 'Run Blocking Grade']].copy().sort_values('blocking grade', ascending=False).reset_index(drop=True)
teams_2024

,Tm,Points For,Points Against,blocking grade,Pass Blocking Grade,Run Blocking Grade
0,Atlanta Falcons,389,423,77.80,72.8,82.8
1,San Francisco 49ers,389,436,76.00,72.0,80.0
2,Denver Broncos,425,311,75.65,83.6,67.7
3,Indianapolis Colts,377,427,74.75,72.5,77.0
4,Philadelphia Eagles,463,303,74.55,73.9,75.2
5,Tampa Bay Buccaneers,502,385,74.10,82.5,65.7
6,Detroit Lions,564,342,73.40,68.4,78.4
7,Kansas City Chiefs,385,326,72.15,70.7,73.6
8,Green Bay Packers,460,338,72.10,82.1,62.1
9,Arizona Cardinals,400,379,71.20,75.8,66.6
